# **RFM Analysis**

Veri seti İngiltere merkezli online bir satış mağazasının 01/12/2009–09/12/2011 tarihleri arasındaki satışlarını içeriyor.

In [1]:
import pandas as pd
import numpy as np

In [10]:
pip install openpyxl

In [14]:
df = pd.read_csv('/content/online_retail_II.csv')

In [15]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom


In [22]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

Description ve customer ID değerlerinde boş satırlar olduğunu görüyoruz. Bu boş değerleri analizimiz için şuan siliyoruz.

In [23]:
df.dropna(inplace = True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      417534 non-null  object 
 1   StockCode    417534 non-null  object 
 2   Description  417534 non-null  object 
 3   Quantity     417534 non-null  int64  
 4   InvoiceDate  417534 non-null  object 
 5   Price        417534 non-null  float64
 6   Customer ID  417534 non-null  float64
 7   Country      417534 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 28.7+ MB


Burada customer ID alanı float gözüküyor fakat bu alanın ınt olması gerekmektedir. Bu nedenle bu alanı dönüştürmek ile başlayabiliriz.

In [25]:
df['Customer ID'] = df['Customer ID'].astype(int)

In [48]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [41]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  int64         
 7   Country      417534 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 28.7+ MB


# **Recency**

In [50]:
df['InvoiceDate'].max()
# En son yapılan alışveriş tarihi çağırıldı.

Timestamp('2010-12-09 20:01:00')

In [51]:
import datetime as dt
# Tarih ile işlem yapacağımız için tarih kütüphanesi çağırıldı.

In [52]:
today_date = dt.datetime(2011, 12, 9)
# En son alışveriş yapılan fatura tarihi bugünün değeri gibi atanır. Analiz yapılan tarih atanabilir fakat bu çok sağlıklı bir analiz sunmayacağı için son alışveriş tarihini almak daha doğru olacaktır.


In [53]:
df.groupby('Customer ID').agg({'InvoiceDate':'max'}).head()
# Müşteriler en son yaptığı alışverişin fatura tarihine göre gruplandı.

,InvoiceDate
Customer ID,
12346,2010-10-04 16:33:00
12347,2010-12-07 14:57:00
12348,2010-09-27 14:59:00
12349,2010-10-28 08:23:00
12351,2010-11-29 15:23:00


Recency değerini bulabilmemiz için yapılması gereken tek bir hamle kaldı. O da bugünün tarihinden her bir müşterinin en son yaptığı alışverişin fatura tarihini çıkarmak

In [54]:
purchase_df = (today_date - df.groupby('Customer ID').agg({'InvoiceDate':'max'}))
# Bugünün tarihinden her bir müşterinin son alışveriş tarihi çıkarıldı. Bu da purchase_df adlı bir dataframe’ e atandı.

In [55]:
purchase_df.head()

,InvoiceDate
Customer ID,
12346,430 days 07:27:00
12347,366 days 09:03:00
12348,437 days 09:01:00
12349,406 days 15:37:00
12351,374 days 08:37:00


In [56]:
purchase_df.rename(columns = {'InvoiceDate':'Recency'}, inplace = True)
# Oluşturulmuş dataframe’ de görünen InvoiceDate verileri artık bizim Recency değerlerimizdir. Bu nedenle dataframe’ de de bu alanın ismi “Recency” olarak güncellendi.


In [60]:
purchase_df['Recency'] = purchase_df['Recency'].apply(lambda x: x.days)
# Recency değerinde gün ve saat bilgisi yer almaktadır. Oysa bize sadece gün bilgisi yeterli olacaktır. Bu nedenle apply ve lambda kullanılarak her bir verinin düzeltilmesi sağlanır.

In [61]:
purchase_df.head()

,Recency
Customer ID,
12346,430
12347,366
12348,437
12349,406
12351,374


# **Frequency**

In [63]:
freq_df = df.groupby('Customer ID').agg({'Invoice':'nunique'})
# Her bir müşteriyi alışveriş fatura sayısına göre gruplar.

In [64]:
freq_df.head()

,Invoice
Customer ID,
12346,15
12347,2
12348,1
12349,4
12351,1


In [65]:
freq_df.rename(columns = {'Invoice':'Frequency'}, inplace = True)
# Dataframe’ deki Invoice değerleri artık müşterinin alışveriş sıklığıdır, yani bizim Frequency değerimizdir. Bu nedenle sütun ismi değiştirildi.

In [66]:
freq_df.head()

,Frequency
Customer ID,
12346,15
12347,2
12348,1
12349,4
12351,1


# **Monetary**

In [75]:
df['TotalPrice'] = df['Quantity'] * df['Price']
# Parasal hesaplama yapabilmemiz için önce her bir ürünün ne kadar tuttuğunu hesaplamamız gerekir. Adet ve birim fiyatı çarparak toplam tutar değerleri bulundu.

In [73]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,1000.8
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,972.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,972.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,4838.4
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,720.0


In [76]:
monetary_df = df.groupby('Customer ID').agg({'TotalPrice':'sum'})
# Her bir müşterin alışverişlerinin toplam tutar değerleri hesaplandı.

In [77]:
monetary_df.rename(columns = {'TotalPrice':'Monetary'}, inplace = True)
# Dataframe’ de TotalPrice sütun ismi “Monetary” olarak değiştirildi.

In [78]:
monetary_df.tail()
# Veri setindeki son 5 satırı bize getirir.

,Monetary
Customer ID,
18283,641.77
18284,436.68
18285,427.00
18286,1188.43
18287,2340.61


öylece RFM Analizi için tüm metriklerimizi bulmuş olduk.

In [79]:
#Kontrol ettiğimizde
print(purchase_df.shape)
print(freq_df.shape)
print(monetary_df.shape)

(4383, 1)
(4383, 1)
(4383, 1)


Tüm satır sayılarımız eşit, bir sonraki adıma geçebiliriz.

# **RFM Skorları**

In [81]:
rfm = pd.concat([purchase_df, freq_df, monetary_df], axis = 1)
# Bulduğumuz tüm değerleri bir dataframe’ de birleştirildi. Buradaki axis = 1 sütun bazında birleştirme yapmamız sağlar. (Purchase df recency df anlamına gelmekte)

In [82]:
rfm.tail()

,Recency,Frequency,Monetary
Customer ID,,,
18283,381,6,641.77
18284,428,2,436.68
18285,659,1,427.00
18286,475,3,1188.43
18287,381,5,2340.61


In [83]:
rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels = [5, 4, 3, 2, 1])
rfm["FrequencyScore"] = pd.qcut(rfm["Frequency"].rank(method = "first"), 5, labels = [1, 2, 3, 4, 5])
rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"], 5, labels = [1, 2, 3, 4, 5])
# Yukarıda kodlar ile rfm tablosuna 3 yeni sütun eklendi. "qcut" fonksiyonu sayesinde Recency, Frequency ve Monetary tablosu altında bulunan bütün değerleri 5 ayrı gruba ayrıldı. Yani her bir değer için skor ataması yapıldı diyebiliriz.
# Burada kritik nokta, Recency değerine atanan skordur. Recency değerinin küçük olması bizim için olumlu olduğundan en düşük değere en yüksek değer atanır. Bu nedenle değer ataması diğerlerinin tersine 5' ten 1' e doğru yapılmıştır.
rfm.tail()


,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore
Customer ID,,,,,,
18283,381,6,641.77,4,4,3
18284,428,2,436.68,3,3,2
18285,659,1,427.00,1,2,2
18286,475,3,1188.43,2,3,4
18287,381,5,2340.61,4,4,5


In [84]:
rfm.describe().T


,count,mean,std,min,25%,50%,75%,max
Recency,4383.0,454.395848,98.177019,364.00,380.00,415.00,501.00,737.00
Frequency,4383.0,5.381474,10.051921,1.00,1.00,3.00,6.00,270.00
Monetary,4383.0,1904.679118,8519.369281,-25111.09,285.26,655.94,1645.69,341776.73


**Her bir değerimiz için skor atandıktan sonra bunların RFM skolarını bulmamız gerekmektedir.**

In [88]:
rfm['RFM_Score'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str) + rfm['MonetaryScore'].astype(str)
#Recency, Frequency ve Monetary skorlarını tek tek stringe çevirip topladık ve bize her müşterinin RFM skorlarını vermiş oldu. Integer olarak kalsaydı değerleri toplardı, fakat bize değerlerin yan yana yazılı hali gerekmektedir.
rfm.tail()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score
Customer ID,,,,,,,
18283,381,6,641.77,4,4,3,443
18284,428,2,436.68,3,3,2,332
18285,659,1,427.00,1,2,2,122
18286,475,3,1188.43,2,3,4,234
18287,381,5,2340.61,4,4,5,445


### **RFM Skorlarına göre artık müşterilerimizi segmentlere ayırabiliriz.**

if ile yapabiliriz fakat karmaşık olacağından dolayı regex fonksiyonu kullanılmıştır.

In [90]:
seg_map = {  r'[1-2][1-2]': 'Hibernating',
             r'[1-2][3-4]': 'At Risk',
             r'[1-2]5': 'Can\'t Loose',
             r'3[1-2]': 'About to Sleep',
             r'33': 'Need Attention',
             r'[3-4][4-5]': 'Loyal Customers',
             r'41': 'Promising',
             r'51': 'New Customers',
             r'[4-5][2-3]': 'Potential Loyalists',
             r'5[4-5]': 'Champions'
}

# Regex komutlarını yorumlamak gerekirse, ilk değerler Recency' yi, ikinci değerler Frequency' yi temsil etmektedir. Buna göre ilk kod "müşterimizin R ve F skorları 1 veya 2 değerlerini alıyorsa "Hibernating" segmentine aittir" şeklinde yorumlanabilir. Yine Recency skoru 5, Frequency 4 veya 5 skorunu alıyor ise "Champions" segmentine aittir diyebiliriz.

In [91]:
rfm["Segment"] = rfm["RecencyScore"].astype(str) + rfm["FrequencyScore"].astype(str)
rfm["Segment"] = rfm["Segment"].replace(seg_map, regex = True)
#Bu kod blokları RF skorlarını oluşturup regex komutlarına göre segmentlere ayırır.


In [92]:
rfm.tail()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Score,Segment
Customer ID,,,,,,,,
18283,381,6,641.77,4,4,3,443,Loyal Customers
18284,428,2,436.68,3,3,2,332,Need Attention
18285,659,1,427.00,1,2,2,122,Hibernating
18286,475,3,1188.43,2,3,4,234,At Risk
18287,381,5,2340.61,4,4,5,445,Loyal Customers
